In [1]:
# pip install bitsandbytes accelerate flash-attn

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import torch
import bz2
import pickle
import importlib
from tqdm import tqdm


import time
from IPython.display import JSON
import sys
sys.settrace(None)
import pdb

import copy

import psycopg
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, StoppingCriteria, StoppingCriteriaList, DynamicCache, OffloadedCache #, CodeGenTokenizer
from transformers.generation.logits_process import LogitsProcessorList

In [4]:
assert torch.cuda.is_available()

In [5]:
### Remind to remove '.py'
# index_module = importlib.import_module('phi4_index')
#index_module = importlib.import_module('http_index')
index_module = importlib.import_module('qwen25_index')
#index_module = importlib.import_module('gemma3_index')
#index_module = importlib.import_module('llama_index')
index_config = getattr(index_module, 'index_config')

# model_module = importlib.import_module('phi4_model')
model_module = importlib.import_module('qwen25_1B_model')
#model_module = importlib.import_module('gemma3_4B_model')
#model_module = importlib.import_module('llama_1B_model')
model_config = getattr(model_module, 'model_config')

Loading Qwen/Qwen2.5-1.5B-Instruct


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
streamer = TextStreamer(model_config.tokenizer)

In [7]:
from ctrie import ConstrainedLogitsProcessor, ConstrainedStateList, \
                    ConstrainedState, DictIndex

In [8]:
question = '''Which city is the capital of the country where the Eiffel Tower is?'''
question = '''Who was the quarterback of the team that won Super Bowl 50?'''
question = '''Who is the lead character in the Uncharted series?'''
question = '''Who played Gandalf in the Lord of the Rings movies?'''
question = '''What was Nirvana's last studio album called?''' # TODO test with beamsearch 3. with beam search if I have only 1 allowed token the other beams could generate something illegal and than crash
# TODO understand the crash is due to beam search and abandon those illegal paths. or try to explore always different triples with beam search???


question = '''How many members were there in the original NSYNC music group?'''

question = 'Who influenced Nirvana band?'

question = 'Is Johnny Depp or Brad Pitt older?'
#question = 'Who is the mother of the director of film Polish-Russian War (Film)?' # Małgorzata Braunek
#question = 'Which film came out first, Blind Shaft or The Mask Of Fu Manchu?' # The Mask Of Fu Manchu
question = 'When did John V, Prince Of Anhalt-Zerbst\'s father die?' # 12 June 1516
#question = 'Is Johnny Depp older than Brad Pitt?'

prompted_texts = [model_config.apply_prompt_template(question)]

In [9]:
print(prompted_texts[0])

<|im_start|>system
You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

Process:

    You receive an input question.

    You determine the reasoning path needed to answer the question based on the information available.

    You explicitly list relevant facts, one per line, starting with "Fact:".

    You explain your reasoning step by step and provide a detailed answer, justifying it based on the facts.

    You conclude with a short and concise answer.

Handling missing facts:

    If no relevant facts are found in the knowledge base, you:
    a) Provide an answer based on general knowledge, explicitly stating that no supporting facts were found.
    b) If you do not know the answer or are not confident enough, simply reply: "I don't know."

You must always follow these instructions precisely and ensure your responses adhere strictly to this prompt.
<|im_end|>
<|im_start|>user
Which mountain is taller b

In [10]:
inputs = model_config.tokenizer(prompted_texts, return_tensors='pt', padding=True, padding_side='right')
inputs = inputs.to(model_config.model.device)

In [11]:
model_config.model.device

device(type='cuda', index=0)

In [12]:
torch.cuda.empty_cache()

In [13]:
num_beams = 3

auto_streamer = streamer if num_beams == 1 else None

states = ConstrainedStateList(
    [ConstrainedState(
                begin_pattern = model_config.switch_pattern,
                end_pattern = model_config.newline_token,
                cache_index = DictIndex(end_of_triple=index_config.index.end_of_triple),
                subtree_cache = DictIndex(end_of_triple=index_config.index.end_of_triple),
                oneleaf_cache = DictIndex(end_of_triple=index_config.index.end_of_triple)
            ) for _ in range(num_beams)],
            num_beams=num_beams,
            batch_size = 1,
            pad_token_id = model_config.tokenizer.eos_token_id)

constrained_processor = ConstrainedLogitsProcessor(
    index=index_config.index,
    end_token=model_config.newline_token, states=states, tokenizer=model_config.tokenizer)
logits_processor_list = LogitsProcessorList([
    constrained_processor
])

model_config.model.eval()
start = time.time()

with torch.no_grad():

    #pdb.set_trace()
    out = model_config.model.generate(
        **inputs,
        logits_processor=logits_processor_list,
        max_new_tokens=800,
        streamer = auto_streamer,
        do_sample = False,
        temperature = 0.0,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams // 2,
        ## TODO group beam search works differently with logitsprocessors: input ids shape is group shape = num_beams / num_beam_groups
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        #stopping_criteria=stopping_criteria,
        use_cache=True,
        #past_key_values=past_key_values,
        kwargs = {'constrained_state': states}, # passing state
    )

print('Elapsed', time.time() - start)

/opt/conda/envs/gemma3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/envs/gemma3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Elapsed 34.7655770778656


In [14]:
torch.cuda.empty_cache()

In [15]:
_from = len(inputs.input_ids[0]) # 0
for i in range(out.shape[0]):
    print('-'*30, sum(out[i][_from:]), len(out[i][_from:]))
    print(model_config.tokenizer.decode(out[i][_from:]))

------------------------------ tensor(1007372, device='cuda:0') 161
Reasoning: To answer this question, I need to find the date of death of John V, Prince of Anhalt-Zerbst's father.
Fact: <John V, Prince of Anhalt-Zerbst> <father> <Ernest I, Prince of Anhalt-Dessau> .
Fact: <Ernest I, Prince of Anhalt-Dessau> <date of death> <1516-06-12T00:00:00Z> .
Long answer: The father of John V, Prince of Anhalt-Zerbst, Ernest I, Prince of Anhalt-Dessau, died on June 12, 1516.

Answer: June 12, 1516.<|im_end|>
------------------------------ tensor(1158718, device='cuda:0') 161
Reasoning: To answer this question, I need to find the date of death of John V, Prince of Anhalt-Zerbst's father.
Fact: <John V, Prince of Anhalt-Zerbst> <father> <Ernest I, Prince of Anhalt-Dessau> .
Fact: <Ernest I, Prince of Anhalt-Dessau> <date of death> <1516-06-12T00:00:00Z> .
Long answer: John V, Prince of Anhalt-Zerbst's father, Ernest I, Prince of Anhalt-Dessau, died on June 12, 1516.

Answer: June 12, 1516.<|im_end

In [16]:
for i, triple in enumerate(states[0].generated_triples):
    print(i, model_config.tokenizer.decode(triple)[:-1], triple, end='\n')

0  <Johnny Depp> <date of birth> <1963-06-09T00:00:00Z> . [366, 87860, 1581, 602, 29, 366, 1028, 315, 7194, 29, 366, 16, 24, 21, 18, 12, 15, 21, 12, 15, 24, 51, 15, 15, 25, 15, 15, 25, 15, 15, 57, 29, 659, 198]
1  <Brad Pitt> <date of birth> <1963-12-18T00:00:00Z> . [366, 61781, 21139, 29, 366, 1028, 315, 7194, 29, 366, 16, 24, 21, 18, 12, 16, 17, 12, 16, 23, 51, 15, 15, 25, 15, 15, 25, 15, 15, 57, 29, 659, 198]


# Debug unconstrained

In [16]:
num_beams = 1

auto_streamer = streamer if num_beams == 1 else None

model_config.model.eval()
start = time.time()

with torch.no_grad():

    #pdb.set_trace()
    out = model_config.model.generate(
        **inputs,
        #logits_processor=logits_processor_list,
        max_new_tokens=800,
        streamer = auto_streamer,
        do_sample = False,
        temperature = 0.0,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams // 2,
        ## TODO group beam search works differently with logitsprocessors: input ids shape is group shape = num_beams / num_beam_groups
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        #stopping_criteria=stopping_criteria,
        use_cache=True,
        #past_key_values=past_key_values,
        #kwargs = {'constrained_state': states}, # passing state
    )

Loading Qwen/Qwen2.5-1.5B-Instruct
<|im_start|>system
You are a helpful question-answering assistant that bases its answers on facts from a knowledge base and always respects the prompt.

Process:

    You receive an input question.

    You determine the reasoning path needed to answer the question based on the information available.

    You explicitly list relevant facts, one per line, starting with "Fact:".

    You explain your reasoning step by step and provide a detailed answer, justifying it based on the facts.

    You conclude with a short and concise answer.

Handling missing facts:

    If no relevant facts are found in the knowledge base, you:
    a) Provide an answer based on general knowledge, explicitly stating that no supporting facts were found.
    b) If you do not know the answer or are not confident enough, simply reply: "I don't know."

You must always follow these instructions precisely and ensure your responses adhere strictly to this prompt.
<|im_end|>
<|im_sta